# External Trigger Demonstration Notebook

## Aims of Notebook

- Show how the external trigger may be used

## To Do

- Tidy notebook

In [2]:
# Import libraries
%load_ext autoreload
%autoreload 2

from qick import *
import Pyro4
%pylab inline

# Pyro communicates with board
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

# Static IP proxy
ns_host = "192.168.2.99" # <-- SET IP ADDRESS
ns_port = 8888
proxy_name = "myqick"

# QICK object
ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib

QICK configuration:

	Board: RFSoC4x2

	Software version: 0.2.267
	Firmware timestamp: Wed Sep  6 18:49:29 2023

	Global clocks (MHz): tProcessor 409.600, RF reference 491.520

	2 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 0, blk 0 is DAC_B
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 2, blk 0 is DAC_A

	2 readout channels:
	0:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by output 7, pin 14, feedback

In [4]:
prog = QickProgram(soccfg)

# Set PMOD0_0 output high
prog.regwi(0, 4, 1)
prog.seti(7, 0, 4, 0)

# Generate DAC_A signal
prog.declare_gen(ch=1, nqz=1)
prog.set_pulse_registers(ch=1, gain=10000, freq=prog.freq2reg(100), phase=0, style="const", length=10000)
prog.pulse(ch=1, t=0)

prog.wait_all()

# Increment counter each time program is run
# prog.memri(0, 1, 1)
# prog.mathi(0, 1, 1, "+", 1)
# prog.memwi(0, 1, 1)

prog.end()

# prog.config_all(soc)
prog.compile()

# soc.tproc.single_write(addr=1, data=0)
# soc.tproc.single_write(addr=2, data=0)

t0 = time.time() 
soc.start_src("external")
# soc.start_src("internal")
soc.load_bin_program(prog.binprog)
soc.tproc.start()

# time.sleep(5)

# counts = np.int32(soc.tproc.single_read(addr=1))
# trg_word_lower = np.int32(soc.tproc.single_read(addr=2))
# trg_word_upper = np.int32(soc.tproc.single_read(addr=3))

# print("%i counts in %i seconds" % (counts, time.time() - t0))
# print("trigger, lower 32 bit word: %i, upper 32 bit word: %i" % (trg_word_lower, trg_word_upper))